In [ ]:
# installing libraries 
!pip install wikipedia
!pip install transformers
!pip install nltk
!pip install fuzzywuzzy
!pip install rdflib
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=aeed12725f517c04d09cbadc7352e2fa6c3e4556fc4ab519175ae34d071a8049
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.9 MB/s 
     |████████████████████████████████| 120 kB 48.3 MB/s 
     |████████████████████████████████| 6.6 MB 10.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 

In [ ]:
# Install spaCy and neuralcoref with correct versions for neural coref
!pip install -U spacy==2.1.0 
!python -m spacy download en
!pip uninstall -y neuralcoref 
!pip install neuralcoref --no-binary neuralcoref

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 27.7 MB 1.3 MB/s 
     |████████████████████████████████| 3.2 MB 48.7 MB/s 
     |████████████████████████████████| 82 kB 454 kB/s 
     |████████████████████████████████| 184 kB 67.2 MB/s 
     |████████████████████████████████| 2.1 MB 36.3 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.4
    Uninstalling srsly-2.4.4:
      Successfully uninstalled srsly-2.4.4
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.7
    Uninstalling preshed-3.0.7:
      Successfully uninstalled preshed-3.0.7
  Attempting uninstall: blis
    Found existing installation: blis 0.7.8
    Uninstalling blis-0.7.8:
      Successfully uninstalled blis-0.7.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting unin

In [ ]:
#importing libraries
import nltk
from nltk import sent_tokenize 
import wikipedia
import spacy
import neuralcoref
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import requests
from bs4 import BeautifulSoup
import urllib.parse
from urllib.request import urlopen
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import re
import pandas as pd
from fuzzywuzzy import fuzz
import ast

100%|██████████| 40155833/40155833 [00:01<00:00, 26506044.94B/s]
/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#loading
nltk.download("punkt")
nlp = spacy.load("en_core_web_sm")
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
neuralcoref.add_to_pipe(nlp)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
url_link = "https://en.wikipedia.org/wiki/Berlin_Wall"
article_name = url_link[30:] #https://en.wikipedia.org/wiki/{article_name}

article_name_result = wikipedia.page(wikipedia.search(article_name)[0], auto_suggest=False)
article_name_content = article_name_result.content
without_article_name_content = article_name_content.replace("\n", "").replace("\t", "")

page = urlopen(url_link)
html_bytes = page.read()
html = html_bytes.decode("utf-8")

    
def get_soup(url):
    response = requests.get(url)
    if response.ok:
        return BeautifulSoup(response.text, features="html.parser")

def get_links(soup):
    links = []
    for tag in soup.findAll("a", href=True):    
        links.append(dict(url=tag.get("href"), text=tag.text))
    return links

def Diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

# soup = get_soup('www.example.com')
soup = BeautifulSoup(html, features="html.parser")
links = get_links(soup)
# of form <a href="www.yy.com">text</a> 

sparql = SPARQLWrapper('https://dbpedia.org/sparql')
sparql.setQuery(f'''
     select ?link ?pred ?pred_inv where {{
    <http://dbpedia.org/resource/{article_name}> <http://dbpedia.org/ontology/wikiPageWikiLink> ?link .
    optional {{
        <http://dbpedia.org/resource/{article_name}> ?pred ?link
        filter(?pred != <http://dbpedia.org/ontology/wikiPageWikiLink>)
    }}
    optional {{
        ?link ?pred_inv <http://dbpedia.org/resource/{article_name}>
        filter(?pred_inv != <http://dbpedia.org/ontology/wikiPageWikiLink>)
    }}
}} order by ?link

''')
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()

#key_to_lookup = 'pred'
#if key_to_lookup in qres:
#pprint(qres)


result = qres['results']['bindings']
m = len(result)
#print(result[0]['link']['value'])
c = []
for i in range (m):
    #print(result[i])
    #here 28 is removing https://dbpedia/resource but then File and categories would still be there
    x = result[i]['link']['value']
    
    #c.append(x)
    regex = re.compile('/File:')
    regex1 = re.compile('/Category:')
     #removinng files and categories   
    if(regex.search(x) == None and regex1.search(x) == None):
        c.append(x)
   
#print(c)

list_wiki_ids = [w[28:] for w in c]
string_wikiHyperlinks = '/wiki/'
#converting dbpedia link in /wiki/* format
list2 = (string_wikiHyperlinks + pd.Series(list_wiki_ids)).tolist()
list_1 = [] #containing duplicate values too
for i in range(len(links)):
    val = links[i]['url']
    if any(val in x for x in list2) == True:
        list_1.append(val)

mylist = list(dict.fromkeys(list_1))
a_3 = Diff(mylist, list2)

list_ax = []
for i in range(len(links)):
    val_0 = links[i]['url']
    al_9 = urllib.parse.unquote(val_0)
    if any(al_9 in x for x in a_3) == True:
        list_ax.append(al_9)

mylistax = list(dict.fromkeys(list_ax))

final_list = mylistax + mylist 

bn = [i for i in links if i["url"] in final_list] #dict of url nd corresponding texts
#print(bn)

unique_dict = [dict(t) for t in {tuple(d.items()) for d in bn}]
df_unique_dict = pd.DataFrame(unique_dict)
end_df = df_unique_dict[df_unique_dict.columns.repeat(2)]
end_df.columns = [f'{a}{b}' for a in df_unique_dict for b in ('', '_2')]
end_df.columns = ['url_head', 'url_tail', 'head', 'tail']

list_text = []
for i in bn:
    list_text.append(i['text'])
#print(list_text)


list_fuzzymatch_hyperlinks = []
for i in range(len(list_text)):
      # This is 0
# Score for fuzzy matching after tokenization
    if fuzz.partial_token_set_ratio(list_text[i], without_article_name_content) == 100:
          list_fuzzymatch_hyperlinks.append(list_text[i]) 

unique_list_fuzzymatch_hyperlinks = list(set(list_fuzzymatch_hyperlinks))
#print(list_fuzzymatch_hyperlinks)

In [ ]:
'''
doc = nlp(without_article_name_content)
for token in doc:
    if token.pos_ == "PRON":
      pass
      #print(token.text, "||", token.pos_)
print("Pronouns are present, Do coref resolution")
'''

def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

# Retrieve a list of all the clusters of corefering mentions using the doc._.coref_clusters attribute 
#print(doc._.coref_clusters)

# Replace corefering mentions with the main mentions in each cluster by using the doc._.coref_resolved attribute.
coref_resolved_text = coref_resolution(without_article_name_content)
#print(coref_resolved_text)

In [ ]:
#coref_resolved_text

'The Berlin Wall (German: Berliner Mauer, pronounced [bɛʁˌliːnɐ ˈmaʊ̯ɐ] (listen)) was a guarded concrete barrier that physically and ideologically divided Berlin from 1961 to 1989 as well as encircling and separating West Berlin from East German territory. Construction of The Berlin Wall was commenced by the German Democratic Republic (GDR, East Germany) on 13 August 1961. The Wall cut off West Berlin from surrounding East Germany, including East Berlin. The barrier included guard towers placed along large concrete walls, accompanied by a wide area (later known as the "death strip") that contained anti-vehicle trenches, beds of nails and other defenses. The Eastern Bloc portrayed the Wall as protecting The Eastern Bloc population from fascist elements conspiring to prevent the "will of the people" from building a socialist state in East Germany. GDR authorities officially referred to the Berlin Wall as the Anti-Fascist Protection Rampart (German: Antifaschistischer Schutzwall, pronounc

In [ ]:
sent_tokenize = sent_detector.tokenize(coref_resolved_text.strip())
#print(len(sent_tokenize))
##print("'\n-----\n'".join(sent_detector.tokenize(berlin_wall_content.strip())))
#print(sent_tokenize) #have to remove headings from sentences

In [ ]:
#sent_tokenize

['The Berlin Wall (German: Berliner Mauer, pronounced [bɛʁˌliːnɐ ˈmaʊ̯ɐ] (listen)) was a guarded concrete barrier that physically and ideologically divided Berlin from 1961 to 1989 as well as encircling and separating West Berlin from East German territory.',
 'Construction of The Berlin Wall was commenced by the German Democratic Republic (GDR, East Germany) on 13 August 1961.',
 'The Wall cut off West Berlin from surrounding East Germany, including East Berlin.',
 'The barrier included guard towers placed along large concrete walls, accompanied by a wide area (later known as the "death strip") that contained anti-vehicle trenches, beds of nails and other defenses.',
 'The Eastern Bloc portrayed the Wall as protecting The Eastern Bloc population from fascist elements conspiring to prevent the "will of the people" from building a socialist state in East Germany.',
 'GDR authorities officially referred to the Berlin Wall as the Anti-Fascist Protection Rampart (German: Antifaschistischer

In [ ]:
#REBEL model extracts both entities and relations
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 10,
    "num_return_sequences": 10,
}
list_triples = []
# Text to extract triplets from
for i in range(10): #len(sent_tokenize)
  text = sent_tokenize[i]



# Tokenizer text
  model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')

# Generate
  generated_tokens = model.generate(
    model_inputs["input_ids"].to(model.device),
    attention_mask=model_inputs["attention_mask"].to(model.device),
    **gen_kwargs,
)

# Extract text
  decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

# Extract triplets
  l1 = []
  for idx, sentence in enumerate(decoded_preds):
      #print(f'Prediction triplets sentence {idx}')
      #print(extract_triplets(sentence))
      l1 += extract_triplets(sentence)
  
  d1 = {}
  ctr = 0
  
  
  for x in l1:
    ctr += 1
    if not str(x) in d1:
        d1[str(x)] = 0
    d1[str(x)] += 1

  for x in d1:
    t = x.replace("}", "")
    final_dict = t + ", 'Confidence': " + str(d1[x]/ctr) + "}"
    #print(final_dict)
    final_dictionary = ast.literal_eval(final_dict)
    list_triples.append(final_dictionary)
    
  
print(list_triples)
    
#with open('TripleDictionaries.txt','w') as data: 
  #data.write(str(list_triples))
  #print("\n\n---------Here starts the", (i+1),"th iteration------------\n\n")
  #print("The sentence is:===>", sent_tokenize[i+1] )

[{'head': 'Berlin Wall', 'type': 'instance of', 'tail': 'guarded concrete barrier', 'Confidence': 0.5789473684210527}, {'head': 'Berlin Wall', 'type': 'instance of', 'tail': 'concrete barrier', 'Confidence': 0.2631578947368421}, {'head': 'Berlin Wall', 'type': 'located in the administrative territorial entity', 'tail': 'Berlin', 'Confidence': 0.10526315789473684}, {'head': 'East German', 'type': 'capital', 'tail': 'Berlin', 'Confidence': 0.05263157894736842}, {'head': 'The Berlin Wall', 'type': 'inception', 'tail': '13 August 1961', 'Confidence': 0.25}, {'head': 'The Berlin Wall', 'type': 'country', 'tail': 'German Democratic Republic', 'Confidence': 0.25}, {'head': 'Berlin Wall', 'type': 'country', 'tail': 'German Democratic Republic', 'Confidence': 0.125}, {'head': 'The Berlin Wall', 'type': 'country', 'tail': 'East Germany', 'Confidence': 0.25}, {'head': 'Berlin Wall', 'type': 'inception', 'tail': '13 August 1961', 'Confidence': 0.125}, {'head': 'East Germany', 'type': 'contains adm

In [ ]:
matched_triples = [i for i in list_triples if ((i["head"] in list_fuzzymatch_hyperlinks) or (i["tail"] in list_fuzzymatch_hyperlinks))]
df1 = pd.DataFrame(matched_triples)
head_url_table = df1.merge(end_df, on='head', how='left')
new_df = head_url_table.drop(['url_tail', 'tail_y'], axis=1).rename({'tail_x':'tail'}, axis = 1)
tail_head_url_table = new_df.merge(end_df, on='tail', how='left')
new_df_head_tail = tail_head_url_table.drop(['url_head_y', 'head_y'], axis=1).rename({'head_x':'head', 'url_head_x': 'url_head'} ,axis = 1)
new_df_head_tail = new_df_head_tail[['head', 'url_head', 'type', 'tail', 'url_tail', 'Confidence']]

In [ ]:
new_df_head_tail

,head,url_head,type,tail,url_tail,Confidence
0,Berlin Wall,NaN,located in the administrative territorial entity,Berlin,/wiki/Berlin,0.105263
1,East German,/wiki/East_Germany,capital,Berlin,/wiki/Berlin,0.052632
2,The Berlin Wall,/wiki/The_Berlin_Wall_(video_game),inception,13 August 1961,NaN,0.250000
3,The Berlin Wall,/wiki/The_Berlin_Wall_(video_game),country,German Democratic Republic,/wiki/East_Germany,0.250000
4,Berlin Wall,NaN,country,German Democratic Republic,/wiki/East_Germany,0.125000
5,The Berlin Wall,/wiki/The_Berlin_Wall_(video_game),country,East Germany,/wiki/East_Germany,0.250000
6,The Berlin Wall,/wiki/The_Berlin_Wall_(video_game),country,East Germany,/wiki/Peaceful_Revolution,0.250000
7,East Germany,/wiki/East_Germany,contains administrative territorial entity,East Berlin,/wiki/East_Berlin,0.187500
8,East Germany,/wiki/Peaceful_Revolution,contains administrative territorial entity,East Berlin,/wiki/East_Berlin,0.187500
9,East Berlin,/wiki/East_Berlin,located in the administrative territorial entity,East Germany,/wiki/East_Germany,0.187500
